In [13]:
# Import libraries
import pandas as pd
import pickle
from pathlib import Path
from datetime import date


In [2]:


# Convert month end prices to monthly percentage change and drop na values

#all_funds_df = all_funds_df.pct_change().dropna()

NameError: name 'all_funds_df' is not defined

In [3]:
#all_funds_df.tail()

In [4]:
""" fidelity_index_focused_models

parameters (from ML model)- investment_amount, investment_date

Transactions dataframe from start date to today with pct change for relevant funds

Investing/Rebalancing function

YEAR 1
year start date - lookup annual contribution amount / investment amount (for ML model, this will be 1, 2, or 3 month of contributions), 
divide by portfolio weights and create row 1

for each row in range of other df 
cumulative return until next 01-01

if date = 01-01

lookup balance at previous day
lookup and add contribution limit 

total divide by asset allocation (FOR ML this happens every month or investment date)

replace row with new amounts

Create a starting code then a loop through the months?

# Set starting values for 2012 based on weight
portfolio_start = 

# Check total = 100
portfolio_sum = portfolio_start.sum()
print(f"Calcalation check: The portfolio starting value is ${portfolio_sum}.")

# Create new dataframe for 2012 return calc using the 2012 start values
portfolio_return_2015 = pd.DataFrame(portfolio_start_2015)
portfolio_return_2015 = portfolio_return_2015.transpose()

# Rename weighted value index to start date 01-01-2015
portfolio_return_2015 = portfolio_return_2015.reset_index(drop=True)
portfolio_return_2015 = portfolio_return_2015.rename(index={0:'2015-01-01'})

# Append the all_etf_df monthly returns
portfolio_return_2015 = portfolio_return_2015.append(all_etf_df.loc['2015-02-01':'2015-12-31']).sort_index()

# Calculate cumulative return and drop na row
portfolio_return_2015 = (1 + portfolio_return_2015.shift(1)).cumprod().dropna(how='all')

# Drop ETFs not included in portfolio for 2015
portfolio_return_2015 = portfolio_return_2015.dropna(axis=1)

# Combine return for portfolio 
portfolio_return_2015 = portfolio_return_2015.sum(axis=1)

# Review the Series
portfolio_return_2015
"""


' fidelity_index_focused_models\n\nparameters (from ML model)- investment_amount, investment_date\n\nTransactions dataframe from start date to today with pct change for relevant funds\n\nInvesting/Rebalancing function\n\nYEAR 1\nyear start date - lookup annual contribution amount / investment amount (for ML model, this will be 1, 2, or 3 month of contributions), \ndivide by portfolio weights and create row 1\n\nfor each row in range of other df \ncumulative return until next 01-01\n\nif date = 01-01\n\nlookup balance at previous day\nlookup and add contribution limit \n\ntotal divide by asset allocation (FOR ML this happens every month or investment date)\n\nreplace row with new amounts\n\nCreate a starting code then a loop through the months?\n\n# Set starting values for 2012 based on weight\nportfolio_start = \n\n# Check total = 100\nportfolio_sum = portfolio_start.sum()\nprint(f"Calcalation check: The portfolio starting value is ${portfolio_sum}.")\n\n# Create new dataframe for 2012

In [8]:
# This function sets up the new holding account with a starting balance of the IRA contribution amount for the year
# This function also creates an investment account with a portfolio attached for investment and rebalancing

# Data start date 2012-01-01

portfolio_investment_start_date = '2013-01-01'

portfolio_selection = 'fidelity_index_focused_models'

# Read in portfolio data from pkl file

pkl_file = open(Path(f"./Resources/{portfolio_selection}.pkl"), 'rb')
dataframe = pickle.load(pkl_file)
pkl_file.close()

# create new account with the portfolio asset allocation as the started 'balance'
new_account = dataframe

# Read in ira contribution data from pkl file

pkl_file = open(Path(f"./Resources/ira_contribution_limits.pkl"), 'rb')
dataframe = pickle.load(pkl_file)
pkl_file.close()

# Locate the contribution limit for the portfolio start date
ira_contribution = dataframe.loc[portfolio_investment_start_date]['ira_limit'][0]

# Fund a new holding account with the ira_contribution amount for the year
holding_account = ira_contribution

# This amount will be fed in from the ML model
investment_amount = 2000

# Validate investment amount and update holding account balance
if investment_amount <= holding_account:
    # update holding account balance
    holding_account = holding_account - investment_amount
    validated_amount = investment_amount
    
else: 
    validated_amount = holding_account
    holding_account = 0

new_account = round(validated_amount * new_account/100, 2)
display(new_account)
display(f'The holding account balance is now ${holding_account}.')

,FXAIX,FSMDX,FSSNX,FSGGX,FXNAX,FCONX,BIL
2013-01-01,700.0,60.0,80.0,360.0,700.0,60.0,40.0


'The holding account balance is now $3500.'

In [91]:
# Maintenance of account
# append day by day performance
# trigger contribution dates to top up by investment_amount and rebalance

# This function aggregates the portfolio value by the fund return amount,
# unless a contribution is made.
# If a contribution is made, the amount is added to the portfolio and the portfolio is rebalanced back to target weights

today = date.today()

# Dummy data for investment dates
investment_dates = pd.DataFrame(
    index=['2013-06-01','2013-08-01'],
    data=[1,3],
    columns=['# Investment Months']
)


# Note - freq 'MS' = month start

# create investment date range
inv_date_range = pd.date_range(start=portfolio_investment_start_date,end=today, freq='D').tolist()

# pull dates from timestamp and convert to list
inv_date_range = [i.strftime('%Y-%m-%d') for i in inv_date_range]

# for each day from portfolio start until investment date
# append row, lookup fund return & multiply by shift(1)

transactions_df = pd.DataFrame(columns=['Amount'])

for day in inv_date_range:
    try:
        # Look up investment dates df and return the number of months of investment
        if investment_dates.loc[day][0]:
            investment_amount = round(investment_dates.loc[day][0] * (ira_contribution / 12), 2)
            df = pd.DataFrame(index = [day], data = [investment_amount], columns = ['Amount'])
            transactions_df = transactions_df.append(df, ignore_index=False)
            # append row to new_account df
            # create separate function to rebalance?
    except:
        # If not an investment date, look up the return for that day
        # increase fund value by return
        pass
        
transactions_df


,Amount
2013-06-01,458.33
2013-08-01,1375.00
